
# Tinkoff sirius ML 2023 RecSys intro task (Модели)

## 🛑 Attention! 🛑

Дорогие проверяющие, я отправил работу до 1-го сентрябре, после узнал, что дедлайн перенесли, пожалуйста, вернитесь сюда после нового дедлайна (8-го сентября), это того стоит.

### Бибилиотеки, данные, метрики, класс для наследования

In [ ]:
!pip install implicit >> /dev/null

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pandas.api.types import CategoricalDtype
from scipy import sparse
import implicit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import ndcg_score

In [ ]:
data_folder = '/content/drive/MyDrive/kion_data_featured/'

users_df = pd.read_csv(data_folder + 'users.csv')
items_df = pd.read_csv(data_folder + 'items.csv')
train_part = pd.read_csv(data_folder + 'train.csv', parse_dates=["last_watch_dt"])
test_part = pd.read_csv(data_folder + 'test_data.csv')
test_part = test_part.groupby("user_id").agg({"ground_truth": list}).reset_index()

In [ ]:
 # https://github.com/deethereal/tinkoff-sirius-ml-2023-recsys-intro-task/blob/main/intro_task.ipynb
 # ACHTUNG! DO NOT TOUCH

def ndcg_metric(gt_items: np.ndarray, predicted: np.ndarray) -> float:
    at = len(predicted)
    relevance = np.array([1 if x in predicted else 0 for x in gt_items])
    # DCG uses the relevance of the recommended items
    rank_dcg = dcg(relevance)
    if rank_dcg == 0.0:
        return 0.0

    # IDCG has all relevances to 1 (or the values provided), up to the number of items in the test set that can fit in the list length
    ideal_dcg = dcg(np.sort(relevance)[::-1][:at])

    if ideal_dcg == 0.0:
        return 0.0

    ndcg_ = rank_dcg / ideal_dcg

    return ndcg_


def dcg(scores: np.ndarray) -> float:
    return np.sum(
        np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)), dtype=np.float64
    )


def recall_metric(gt_items: np.ndarray, predicted: np.ndarray) -> float:
    n_gt = len(gt_items)
    intersection = len(set(gt_items).intersection(set(predicted)))
    return intersection / n_gt


def evaluate_recommender(df: pd.DataFrame, model_preds_col: str, gt_col: str = "ground_truth") -> Dict[str, float]:
    metric_values = []

    for _, row in df.iterrows():
        metric_values.append(
            (ndcg_metric(row[gt_col], row[model_preds_col]), recall_metric(row[gt_col], row[model_preds_col]))
        )

    return {"ndcg": np.mean([x[0] for x in metric_values]), "recall": np.mean([x[1] for x in metric_values])}

In [ ]:
# https://github.com/deethereal/tinkoff-sirius-ml-2023-recsys-intro-task/blob/main/intro_task.ipynb
class BaseRecommender(ABC):
    def __init__(self):
        self.trained = False

    @abstractmethod
    def fit(self, df: pd.DataFrame) -> None:
        # реализация может быть любой, никаких ограничений

        # не забудьте про
        self.trained = True

    @abstractmethod
    def predict(self, df: pd.DataFrame, topn: int = 10) -> List[np.ndarray]:
        # реализация может быть любой, НО
        # должен возвращать список массивов из item_id, которые есть в `item_df`, чтобы корректно работал подсчет метрик
        pass

## Cтатистически подходы

### Топ популярных (Baseline)

In [ ]:
# https://github.com/deethereal/tinkoff-sirius-ml-2023-recsys-intro-task/blob/main/intro_task.ipynb
class TopPopular(BaseRecommender):
    def __init__(self):
        super().__init__()

    def fit(self, df: pd.DataFrame, item_id_col: str = "item_id") -> None:
        # считаем популярность айтемов
        self.recommendations = df[item_id_col].value_counts().index.values
        self.trained = True

    def predict(self, df: pd.DataFrame, topn: int = 10) -> np.ndarray:
        assert self.trained
        # возвращаем для всех одно и то же
        return [self.recommendations[:topn]] * len(df)


toppop = TopPopular()
toppop.fit(train_part)
test_part["toppopular_recs"] = toppop.predict(test_part)
evaluate_recommender(df=test_part, model_preds_col="toppopular_recs")

{'ndcg': 0.17037237918248196, 'recall': 0.0763696799665908}

### Топ популярных для разных категорий

In [ ]:
class CategoryTopPopularRecommender(BaseRecommender):
    def __init__(self, category: str):
        super().__init__()
        self.cat_popularity = None
        self.trained = False
        self.top_popular_items = None
        self.category = category

    def fit(self, train_part: pd.DataFrame, users_df: pd.DataFrame, items_df: pd.DataFrame,
            item_id_col = 'item_id', views_col = 'views', user_id_col='user_id'):
        # Вычисляем общий список популярных фильмов по все значениям категории
        self.top_popular_items = items_df[[item_id_col, views_col]]\
                                  .sort_values(views_col, ascending=False)[item_id_col].to_list()

        # Группируем items_df по категории и сортируем фильмы по популярности (колонка 'views')
        self.cat_popularity = dict(train_part.merge(users_df, how='left', on=user_id_col)[[self.category, item_id_col]]\
                                   .groupby(self.category)[item_id_col]\
                                   .apply(lambda x: x.value_counts().index.tolist()))


        self.trained = True

    def predict(self, test_part: pd.DataFrame, users_df: pd.DataFrame, user_id_col = 'user_id', topn=10):
        assert self.trained

        # Объединяем test_part с users_df по 'user_id' и оставляем только 'user_id' и категорию
        user_data = users_df[[user_id_col, self.category]]
        test_data = test_part.merge(user_data, on='user_id', how='left')

        # Создаем пустой столбец 'recommendations' для хранения рекомендаций
        test_data['recommendations'] = None

        # Для каждой строки в test_data вычисляем рекомендации
        for index, row in test_data.iterrows():
            cat = row[self.category]
            if cat in self.cat_popularity:
                recommendations = self.cat_popularity[cat][:topn]
            # Если мы ничего не знаем про пользователя, то рекомендуем популярное
            else:
                recommendations = self.top_popular_items[:topn]

            test_data.at[index, 'recommendations'] = recommendations

        # Возвращаем столбец 'recommendations' с рекомендациями
        return test_data['recommendations']

for cat in ['sex', 'income', 'age']:
    recommender = CategoryTopPopularRecommender(cat)
    recommender.fit(train_part, users_df, items_df)
    test_part[cat + "_recs"] = recommender.predict(test_part, users_df)
    print(f'Разибение по {cat}: ', evaluate_recommender(df=test_part, model_preds_col=cat + "_recs"))

Разибение по sex:  {'ndcg': 0.1712007695565208, 'recall': 0.07668173604995636}
Разибение по income:  {'ndcg': 0.17148589785741739, 'recall': 0.07695524790918498}
Разибение по age:  {'ndcg': 0.17224145285241338, 'recall': 0.07724899822276024}


Видим, что разбиение по категориям, работает немного лучше, чем бейзлайн.

## Классические методы

### kNN

In [ ]:
class ImplicitRecommender(BaseRecommender):

    def __init__(self, K=15_000):
        super().__init__()
        self.K = K
        self.model = implicit.nearest_neighbours.CosineRecommender(K=K)
        self.user_item_matrix = None

    def fit(self, train_part: pd.DataFrame,
            user_id_col: str = 'user_id', item_id_col: str = 'item_id'):
        self.user_item_matrix = train_part[[user_id_col, item_id_col]].copy()
        self.user_item_matrix['view'] = 1
        self.user_item_matrix = sparse.csr_matrix((self.user_item_matrix['view'].astype(float),
                              (self.user_item_matrix[user_id_col],
                               self.user_item_matrix[item_id_col])))
        self.model.fit(self.user_item_matrix)
        self.trained = True


    def predict(self, test_part: pd.Series, user_id_col = 'user_id', N=10):
        assert self.trained

        return self.model.recommend(test_part[user_id_col].to_list(),
                                    self.user_item_matrix[test_part[user_id_col].to_list()],
                                    filter_already_liked_items = True,
                                    N=N)[0].tolist()


recommender = ImplicitRecommender()
recommender.fit(train_part)
test_part['knn_recs'] =  recommender.predict(test_part)
evaluate_recommender(df=test_part, model_preds_col='knn_recs')

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.012315511703491211 seconds
  warnings.warn(


  0%|          | 0/16519 [00:00<?, ?it/s]

{'ndcg': 0.12234769080835413, 'recall': 0.054391934978456756}

Видим, что качество просело даже по сравнению с бейзлайном.

### TFIDFRecommender

In [ ]:
class TFIDFImplicitRecommender(ImplicitRecommender):

    def __init__(self, K=15_000):
        super().__init__()
        self.model = implicit.nearest_neighbours.TFIDFRecommender(K=K)

recommender = TFIDFImplicitRecommender()
recommender.fit(train_part)
test_part['TFIDF_recs'] =  recommender.predict(test_part)
evaluate_recommender(df=test_part, model_preds_col='TFIDF_recs')

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.013181209564208984 seconds
  warnings.warn(


  0%|          | 0/16519 [00:00<?, ?it/s]

{'ndcg': 0.1272803387919534, 'recall': 0.05654443212247059}

### BM25

Ровно то же самое.

In [ ]:
class BM25ImplicitRecommender(ImplicitRecommender):

    def __init__(self, K=15_000):
        self.model = implicit.nearest_neighbours.BM25Recommender(K=K)


recommender = BM25ImplicitRecommender()
recommender.fit(train_part)
test_part['BM25_recs'] =  recommender.predict(test_part)
evaluate_recommender(df=test_part, model_preds_col='BM25_recs')

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.012892484664916992 seconds
  warnings.warn(


  0%|          | 0/16519 [00:00<?, ?it/s]

{'ndcg': 0.052725947971199026, 'recall': 0.021011723294953752}

### ItemItemRecommender

In [ ]:
class ItemItemImplicitRecommender(ImplicitRecommender):

    def __init__(self, K=15_000):
        super().__init__()
        self.model = implicit.nearest_neighbours.ItemItemRecommender(K=K)


recommenderItemItem = ItemItemImplicitRecommender()
recommenderItemItem.fit(train_part)
test_part['ItemItem_recs'] = recommenderItemItem.predict(test_part)
evaluate_recommender(df=test_part, model_preds_col='ItemItem_recs')

  0%|          | 0/16519 [00:00<?, ?it/s]

{'ndcg': 0.191239213767931, 'recall': 0.08579517402016573}

Item to Item сработал уже лучше, чем разделение на категории.

### BayesianPersonalizedRanking

In [ ]:
class BRPImplicitRecommender(ImplicitRecommender):

    def __init__(self, factors=25, learning_rate=0.01,
                 regularization=0.01, iterations=100):
        super().__init__()
        self.model = implicit.bpr.BayesianPersonalizedRanking(
            factors=factors, learning_rate=learning_rate, regularization=regularization, iterations=iterations, num_threads=0
        )

    def get_params(self, deep=True):
        return {
            "factors": self.model.factors,
            "learning_rate": self.model.learning_rate,
            "regularization": self.model.regularization,
            "iterations": self.model.iterations
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self.model, parameter, value)
        return self


recommender = BRPImplicitRecommender()

param_dist = {
    'factors': [100, 150, 250],
    'learning_rate': [0.01, 0.02, 0.3],
    'regularization': [0.01, 0.03, 0.04],
    'iterations': [100, 250, 500],
}

random_search = RandomizedSearchCV(
    recommender, param_distributions=param_dist,
    n_iter=1, cv=3, random_state=42,
    n_jobs=-1, scoring=ndcg_score
)

random_search.fit(train_part)

best_recommender = random_search.best_estimator_

test_part['BRP_recs'] = best_recommender.predict(test_part)

evaluate_recommender(df=test_part, model_preds_col='BRP_recs')

Для подбора гиперпараметров с помощью кроссвалидации не хватило мощностей. Поэтому лучшая модель это item-to-item

## EDA рекомендаций

Давайте посмотрим, что рекомендуется к гриффинам.

In [ ]:
items_df[items_df['title'] == 'Дуров']

,Unnamed: 0,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,...,actors,description,keywords,views,mean_watched_pct,mean_total_dur,weighted_average_rank,topage,topsex,topincome
2939,2939,6809,film,Дуров,Дуров,2021.0,документальное,Россия,0,16.0,...,NaN,"Уникальная история о лидере нового поколения, ...","Компьютер, Монитор, Гений, Интервью, Предприни...",10208,53.830623,7228.280858,12006.0,age_25_34,М,income_20_40


In [ ]:
simalar_films = recommenderItemItem.model.similar_items(6809)[0].tolist()[:5]

Рекомендованные фильмы к фильму 'Дуров'.

In [ ]:
items_df[items_df['item_id'].isin(simalar_films)]

,Unnamed: 0,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,...,actors,description,keywords,views,mean_watched_pct,mean_total_dur,weighted_average_rank,topage,topsex,topincome
572,572,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,0,18.0,...,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча...",44703,56.394314,24834.558307,12014.0,age_25_34,Ж,income_20_40
1429,1429,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,0,18.0,...,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе...",46131,48.269493,29527.816176,12013.0,age_35_44,Ж,income_20_40
2939,2939,6809,film,Дуров,Дуров,2021.0,документальное,Россия,0,16.0,...,NaN,"Уникальная история о лидере нового поколения, ...","Компьютер, Монитор, Гений, Интервью, Предприни...",10208,53.830623,7228.280858,12006.0,age_25_34,М,income_20_40
4921,4921,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,1,12.0,...,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя...",29403,69.454818,6956.867326,12012.0,age_35_44,М,income_20_40
5064,5064,4151,series,Секреты семейной жизни,Секреты семейной жизни,2021.0,комедии,Россия,0,18.0,...,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо...",21921,43.042881,11124.888463,12009.0,age_25_34,Ж,income_20_40
